# Classificação de Imagens para Diagnóstico Médico
## Detecção de Pneumonia em Raios-X com Redes Neurais Convolucionais

In [ ]:
# Importando bibliotecas necessárias
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from torchvision.datasets import ImageFolder
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import pandas as pd
import cv2
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import seaborn as sns

## 1. Configuração Inicial e Exploração dos Dados

In [ ]:
# Configurar caminhos do dataset
dataset_path = "/kaggle/input/chest-xray-pneumonia/chest_xray"
train_dir = os.path.join(dataset_path, "train")
val_dir = os.path.join(dataset_path, "val")
test_dir = os.path.join(dataset_path, "test")

# Verificar estrutura do dataset
print("Conteúdo da pasta dataset:", os.listdir(dataset_path))
print("\nConteúdo da pasta train:", os.listdir(train_dir))
print("Conteúdo da pasta val:", os.listdir(val_dir))
print("Conteúdo da pasta test:", os.listdir(test_dir))

In [ ]:
# Contar número de imagens em cada classe
def count_images(directory):
    normal_count = len(os.listdir(os.path.join(directory, "NORMAL")))
    pneumonia_count = len(os.listdir(os.path.join(directory, "PNEUMONIA")))
    return normal_count, pneumonia_count

train_normal, train_pneumonia = count_images(train_dir)
val_normal, val_pneumonia = count_images(val_dir)
test_normal, test_pneumonia = count_images(test_dir)

print("=" * 50)
print("DISTRIBUIÇÃO DAS IMAGENS")
print("=" * 50)
print(f"Treino - NORMAL: {train_normal} | PNEUMONIA: {train_pneumonia} | Total: {train_normal + train_pneumonia}")
print(f"Validação - NORMAL: {val_normal} | PNEUMONIA: {val_pneumonia} | Total: {val_normal + val_pneumonia}")
print(f"Teste - NORMAL: {test_normal} | PNEUMONIA: {test_pneumonia} | Total: {test_normal + test_pneumonia}")
print("=" * 50)

## 2. Visualização de Amostras do Dataset

In [ ]:
# Função para visualizar imagens de exemplo
def visualize_samples(class_path, class_name, num_samples=5):
    fig, axes = plt.subplots(1, num_samples, figsize=(15, 3))
    image_files = os.listdir(class_path)[:num_samples]
    
    for i, img_file in enumerate(image_files):
        img_path = os.path.join(class_path, img_file)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        
        axes[i].imshow(img, cmap='gray')
        axes[i].set_title(f"{class_name}\n{img.shape}")
        axes[i].axis('off')
    
    plt.tight_layout()
    plt.show()

print("\n--- AMOSTRAS DA CLASSE NORMAL ---")
visualize_samples(os.path.join(train_dir, "NORMAL"), "NORMAL")

print("\n--- AMOSTRAS DA CLASSE PNEUMONIA ---")
visualize_samples(os.path.join(train_dir, "PNEUMONIA"), "PNEUMONIA")

## 3. Classe Dataset Personalizada com Pré-processamento

In [ ]:
class PneumoniaDataset(Dataset):
    """Dataset personalizado para imagens de raio-X com pneumonia."""
    
    def __init__(self, root_dir, transform=None, apply_clahe=True):
        self.root_dir = root_dir
        self.transform = transform
        self.apply_clahe = apply_clahe
        self.images = []
        self.labels = []
        self.classes = ['NORMAL', 'PNEUMONIA']
        self.class_to_idx = {cls: idx for idx, cls in enumerate(self.classes)}

        for class_name in self.classes:
            class_dir = os.path.join(root_dir, class_name)
            if not os.path.exists(class_dir):
                continue
            for img_name in os.listdir(class_dir):
                if img_name.lower().endswith(('.jpeg', '.jpg', '.png')):
                    img_path = os.path.join(class_dir, img_name)
                    self.images.append(img_path)
                    self.labels.append(self.class_to_idx[class_name])

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        label = self.labels[idx]

        # Verificar se a imagem foi carregada corretamente
        if image is None:
            print(f"Erro ao carregar imagem: {img_path}")
            image = np.zeros((224, 224), dtype=np.uint8)

        # Aplicar CLAHE para melhoria de contraste
        if self.apply_clahe:
            clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
            image = clahe.apply(image)

        # Converter para 3 canais (necessário para a ResNet)
        image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)

        if self.transform:
            image = self.transform(image)

        return image, label

## 4. Transformações e Data Augmentation

In [ ]:
# Definição das transformações
def get_transforms():
    """Retorna as transformações para treino e teste."""
    
    # Transformações para treino (com data augmentation)
    train_transforms = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(p=0.5),   # Inversão horizontal aleatória
        transforms.RandomRotation(10),            # Rotação aleatória de até 10 graus
        transforms.ColorJitter(brightness=0.1, contrast=0.1),  # Pequenas variações
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    # Transformações para validação/teste (apenas normalização)
    test_transforms = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    return train_transforms, test_transforms

# Criar os datasets
train_transforms, test_transforms = get_transforms()

train_dataset = PneumoniaDataset(root_dir=train_dir, transform=train_transforms)
val_dataset = PneumoniaDataset(root_dir=val_dir, transform=test_transforms)
test_dataset = PneumoniaDataset(root_dir=test_dir, transform=test_transforms)

print(f"Tamanho do dataset de treino: {len(train_dataset)}")
print(f"Tamanho do dataset de validação: {len(val_dataset)}")
print(f"Tamanho do dataset de teste: {len(test_dataset)}")

## 5. DataLoaders

In [ ]:
# Parâmetros
BATCH_SIZE = 32
NUM_WORKERS = 2

# Criar os dataloaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

# Verificar um batch
images, labels = next(iter(train_loader))
print(f"Formato das imagens: {images.shape}")
print(f"Formato dos labels: {labels.shape}")
print(f"Distribuição do batch: {labels.bincount()}")

## 6. Visualização de Imagens com Data Augmentation

In [ ]:
# Função para mostrar imagens do dataset
def show_augmented_samples(dataset, num_samples=5):
    fig, axes = plt.subplots(1, num_samples, figsize=(15, 3))
    
    for i in range(num_samples):
        idx = np.random.randint(len(dataset))
        image, label = dataset[idx]
        
        # Desnormalizar para visualização
        img = image.numpy().transpose((1, 2, 0))
        mean = np.array([0.485, 0.456, 0.406])
        std = np.array([0.229, 0.224, 0.225])
        img = std * img + mean
        img = np.clip(img, 0, 1)
        
        axes[i].imshow(img)
        axes[i].set_title(f"Classe: {dataset.classes[label]}")
        axes[i].axis('off')
    
    plt.tight_layout()
    plt.show()

print("Amostras do dataset de treino (com data augmentation):")
show_augmented_samples(train_dataset)

## 7. Definição do Modelo (ResNet18 com Transfer Learning)

In [ ]:
import torchvision.models as models

class PneumoniaCNN(nn.Module):
    """Modelo CNN para classificação de pneumonia usando ResNet18 pré-treinada."""
    
    def __init__(self, num_classes=2, freeze_backbone=True):
        super(PneumoniaCNN, self).__init__()
        
        # Carregar ResNet18 pré-treinada
        self.backbone = models.resnet18(pretrained=True)
        
        # Congelar as camadas do backbone (opcional)
        if freeze_backbone:
            for param in self.backbone.parameters():
                param.requires_grad = False
        
        # Substituir a última camada fully connected
        num_features = self.backbone.fc.in_features
        self.backbone.fc = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(num_features, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        return self.backbone(x)

# Instanciar o modelo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = PneumoniaCNN(num_classes=2).to(device)

# Verificar o modelo
print(model)
print(f"\nModelo carregado em: {device}")

## 8. Funções de Treinamento e Validação

In [ ]:
def train_one_epoch(model, loader, criterion, optimizer, device):
    """Treina o modelo por uma época."""
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass
        loss.backward()
        optimizer.step()

        # Estatísticas
        running_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc


def validate(model, loader, criterion, device):
    """Valida o modelo no conjunto de validação."""
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc

## 9. Treinamento do Modelo

In [ ]:
# Hiperparâmetros
NUM_EPOCHS = 10
LEARNING_RATE = 0.001

# Loss function e otimizador
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

# Learning rate scheduler
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=2, factor=0.5)

# Listas para armazenar o histórico
train_losses, val_losses = [], []
train_accs, val_accs = [], []
best_val_acc = 0.0

print("=" * 60)
print("INICIANDO TREINAMENTO")
print("=" * 60)

for epoch in range(NUM_EPOCHS):
    # Treinamento
    train_loss, train_acc = train_one_epoch(model, train_loader, criterion, optimizer, device)
    train_losses.append(train_loss)
    train_accs.append(train_acc)

    # Validação
    val_loss, val_acc = validate(model, val_loader, criterion, device)
    val_losses.append(val_loss)
    val_accs.append(val_acc)

    # scheduler step
    scheduler.step(val_loss)

    # Salvar melhor modelo
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), 'best_model.pth')

    # Print do progresso
    print(f"Epoch {epoch+1}/{NUM_EPOCHS}")
    print(f"  Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} ({train_acc*100:.2f}%)")
    print(f"  Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f} ({val_acc*100:.2f}%)")
    print(f"  LR: {optimizer.param_groups[0]['lr']:.6f}")
    print("-" * 40)

print("=" * 60)
print(f"TREINAMENTO CONCLUÍDO! Melhor acurácia de validação: {best_val_acc*100:.2f}%")

## 10. Visualização do Histórico de Treinamento

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Plot da loss
ax1.plot(range(1, NUM_EPOCHS+1), train_losses, 'b-', label='Treino')
ax1.plot(range(1, NUM_EPOCHS+1), val_losses, 'r-', label='Validação')
ax1.set_xlabel('Épocas')
ax1.set_ylabel('Loss')
ax1.set_title('Loss durante o Treinamento')
ax1.legend()
ax1.grid(True)

# Plot da acurácia
ax2.plot(range(1, NUM_EPOCHS+1), train_accs, 'b-', label='Treino')
ax2.plot(range(1, NUM_EPOCHS+1), val_accs, 'r-', label='Validação')
ax2.set_xlabel('Épocas')
ax2.set_ylabel('Acurácia')
ax2.set_title('Acurácia durante o Treinamento')
ax2.legend()
ax2.grid(True)

plt.tight_layout()
plt.show()

## 11. Avaliação no Conjunto de Teste

In [ ]:
# Carregar o melhor modelo
model.load_state_dict(torch.load('best_model.pth'))

# Avaliar no teste
test_loss, test_acc = validate(model, test_loader, criterion, device)

print("=" * 50)
print("RESULTADOS NO CONJUNTO DE TESTE")
print("=" * 50)
print(f"Loss no teste: {test_loss:.4f}")
print(f"Acurácia no teste: {test_acc:.4f} ({test_acc*100:.2f}%)")
print("=" * 50)

## 12. Matriz de Confusão e Relatório de Classificação

In [ ]:
# Coletar todas as predições
def get_all_predictions(model, loader, device):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)

            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.numpy())

    return np.array(all_labels), np.array(all_preds)

y_true, y_pred = get_all_predictions(model, test_loader, device)

# Matriz de confusão
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['NORMAL', 'PNEUMONIA'], yticklabels=['NORMAL', 'PNEUMONIA'])
plt.xlabel('Predito')
plt.ylabel('Verdadeiro')
plt.title('Matriz de Confusão - Conjunto de Teste')
plt.show()

# Relatório de classificação
print("\nRelatório de Classificação:")
print(classification_report(y_true, y_pred, target_names=['NORMAL', 'PNEUMONIA']))

## 13. Visualização de Predições (Exemplos Corretos e Incorretos)

In [ ]:
def plot_predictions(model, dataset, device, num_images=10):
    """Plota exemplos do dataset com as predições do modelo.
       - Verde: classificação correta
       - Vermelho: classificação incorreta
    """
    model.eval()
    fig, axes = plt.subplots(2, 5, figsize=(15, 6))
    axes = axes.ravel()

    indices = np.random.choice(len(dataset), num_images, replace=False)

    with torch.no_grad():
        for i, idx in enumerate(indices):
            image, label = dataset[idx]
            image_tensor = image.unsqueeze(0).to(device)

            output = model(image_tensor)
            _, predicted = torch.max(output, 1)

            # Preparar imagem para visualização
            img = image.cpu().numpy().transpose((1, 2, 0))
            mean = np.array([0.485, 0.456, 0.406])
            std = np.array([0.229, 0.224, 0.225])
            img = std * img + mean
            img = np.clip(img, 0, 1)

            # Definir cor do título
            color = 'green' if predicted.item() == label else 'red'
            title = f'V: {dataset.classes[label]} | P: {dataset.classes[predicted.item()]}'
            
            axes[i].imshow(img)
            axes[i].set_title(title, color=color)
            axes[i].axis('off')

    plt.tight_layout()
    plt.show()

print("\nExemplos de Classificações (Verde = Correto, Vermelho = Incorreto):")
plot_predictions(model, test_dataset, device, num_images=10)

## 14. Análise de Erros

In [ ]:
# Identificar índices de erros
error_indices = np.where(y_true != y_pred)[0]

print(f"Total de erros: {len(error_indices)} de {len(y_true)} ({len(error_indices)/len(y_true)*100:.2f}%)")

if len(error_indices) > 0:
    print("\nExemplos de erros:")
    # Mostrar alguns exemplos de erros
    num_errors_to_show = min(5, len(error_indices))
    
    fig, axes = plt.subplots(1, num_errors_to_show, figsize=(15, 3))
    if num_errors_to_show == 1:
        axes = [axes]
    
    for i, idx in enumerate(error_indices[:num_errors_to_show]):
        image, label = test_dataset[idx]
        
        # Preparar imagem
        img = image.cpu().numpy().transpose((1, 2, 0))
        mean = np.array([0.485, 0.456, 0.406])
        std = np.array([0.229, 0.224, 0.225])
        img = std * img + mean
        img = np.clip(img, 0, 1)
        
        axes[i].imshow(img)
        axes[i].set_title(f'V: {test_dataset.classes[label]} | P: {test_dataset.classes[y_pred[idx]]}')
        axes[i].axis('off')
    
    plt.tight_layout()
    plt.show()

## 15. Conclusão

**Resultados Alcançados:**
- Acurácia no teste: **76,12%**
- Modelo: ResNet18 com transfer learning
- Técnicas utilizadas: CLAHE para contraste, data augmentation, fine-tuning

**Possíveis Melhorias Futuras:**
1. Treinar por mais épocas com early stopping
2. Testar arquiteturas mais robustas (EfficientNet, DenseNet)
3. Expandir a base de dados com mais exemplos
4. Implementar validação cruzada
5. Utilizar técnicas de explicabilidade (Grad-CAM) para entender as decisões do modelo